In [12]:
import itertools, collections, csv
from tabulate import tabulate
import numpy as np
from typing import List, Tuple, Dict
import sympy
#init_printing()
my_vars = set(dir())
sympy.init_printing()

In [13]:
y, a, b, c, d, e, f, g = sympy.symbols('y a b c d e f g')
y = a * b

In [14]:
y

In [15]:
class thresholdImplement:
    def __init__(self, num_shares: int, orig_bool) -> None:
        self.num_shares     = num_shares
        self.orig_bool      = orig_bool
        self.i_vars_str     = sorted([x.name for x in orig_bool.free_symbols])
        self.i_vars         = []
        # Due to abitary order of python set variable (orig_bool.free_symbols)
        for i_var_str in self.i_vars_str:
            for i_var in list(orig_bool.free_symbols):
                if i_var_str == i_var.name:
                    self.i_vars.append(i_var)
        self.i_vars_num     = len(self.i_vars)
        self.i_share_vars   = []
        self.i_share_val0   = []
        self.i_share_val1   = []
        self.o_share_vars   = []
        self.io_shares_dict = collections.OrderedDict()
        self.io_counts_dict = collections.OrderedDict()
        self.gen_o_symbols()
        self.gen_i_symbols()
        self.find_i_shares()

    def gen_i_symbols(self):
        for i in range(self.i_vars_num):
            for j in range(self.num_shares):
                exec(f"{self.i_vars[i]}_{j+1} = sympy.symbols('{self.i_vars[i]}_{j+1}')")
                exec(f"self.i_share_vars.append({self.i_vars[i]}_{j+1})")
        globals().update(locals())

    def gen_o_symbols(self):
        for i in range(self.num_shares):
            exec(f"y_{i+1} = sympy.symbols('y_{i+1}')")
            exec(f"self.o_share_vars.append(y_{i+1})")
        globals().update(locals())

    def find_i_shares(self):
        for i in range(2**self.num_shares):
            i_share_bits_str = list(bin(i)[2:].zfill(self.num_shares))
            i_share_bits_int = list(map(int, i_share_bits_str))
            if(np.bitwise_xor.reduce(i_share_bits_int) == 0):
                self.i_share_val0.append(i_share_bits_int)
            else:
                self.i_share_val1.append(i_share_bits_int)

    def check_constraints(self, o_share_vars):
        # assign global symbol to local symbol of class
        for idx, o_share_var in enumerate(o_share_vars):
            self.o_share_vars[idx] = o_share_var

        # generate io_shares_dict for all possible cases
        for i in range(2**self.i_vars_num):
            i_bits_str = bin(i)[2:].zfill(self.i_vars_num)
            i_bits_int = list(map(int, i_bits_str))
            # find correct output
            o_correct = self.orig_bool
            for idx, i_var in enumerate(self.i_vars):
                o_correct = o_correct.subs(i_var, i_bits_int[idx])
            self.io_shares_dict[i_bits_str] = []
            # find available patterns of input shares
            i_avail_shares = [self.i_share_val0 if x == '0' else self.i_share_val1 for x in i_bits_str]
            i_avail_shares_combs = list(itertools.product(*i_avail_shares))
            # calculate ouput share value
            for i_avail_shares_comb in i_avail_shares_combs:
                i_avail_shares_vals = list(itertools.chain.from_iterable(i_avail_shares_comb))
                o_share_vals = []
                for o_share_var in self.o_share_vars:
                    for idx, i_share_var in enumerate(self.i_share_vars):
                        o_share_var = o_share_var.subs(i_share_var, i_avail_shares_vals[idx])
                    o_share_vals.append(o_share_var%2)
                # check correctness
                o_shares_xor = np.bitwise_xor.reduce(o_share_vals)
                if o_correct%2 != o_shares_xor:
                    print(f'Correctness Error: Output {o_correct%2} != {o_shares_xor}')
                self.io_shares_dict[i_bits_str].append([''.join(str(x) for x in i_avail_shares_vals), ''.join(str(x) for x in o_share_vals)])
            # count output share cases
            self.io_counts_dict[i_bits_str] = []
            for j in range(2**self.num_shares):
                o_bits_str = bin(j)[2:].zfill(self.num_shares)
                o_share_case_count = list(itertools.chain.from_iterable(list(self.io_shares_dict[i_bits_str]))).count(o_bits_str)
                self.io_counts_dict[i_bits_str].append(o_share_case_count)
        # draw uniformity table
        print(tabulate(list(ti.io_counts_dict.values()), headers=[bin(x)[2:].zfill(self.num_shares) for x in range(2**self.num_shares)]))

    def write_csv(self):
        with open('ti_table.csv', 'w') as f:
            write = csv.writer(f)
            for i_unshare_val in [bin(x)[2:].zfill(self.i_vars_num) for x in range(self.i_vars_num)]:
                write.writerow([i_unshare_val])
                write.writerows(self.io_shares_dict[i_unshare_val])

In [16]:
ti = thresholdImplement(3, y)
ti.orig_bool

In [17]:
y_1 = a_2*b_2 + a_2*b_3 + a_3*b_2
y_2 = a_3*b_3 + a_1*b_3 + a_3*b_1
y_3 = a_1*b_1 + a_1*b_2 + a_2*b_1

In [18]:
y_1, y_2, y_3

In [20]:
ti.check_constraints([y_1, y_2, y_3])

  000    001    010    011    100    101    110    111
-----  -----  -----  -----  -----  -----  -----  -----
    7      0      0      3      0      3      3      0
    7      0      0      3      0      3      3      0
    7      0      0      3      0      3      3      0
    0      5      5      0      5      0      0      1


In [19]:
#y_1 = a_1 + b_1 #+ c_1
#y_2 = a_2 + b_2 #+ c_2
#y_3 = a_3 + b_3 #+ c_3

In [21]:
ti.write_csv()

In [22]:
ti.o_share_vars[0]

In [23]:
list(ti.io_shares_dict.items())

[('00',
  [['000000', '000'],
   ['000011', '000'],
   ['000101', '000'],
   ['000110', '000'],
   ['011000', '000'],
   ['011011', '110'],
   ['011101', '101'],
   ['011110', '011'],
   ['101000', '000'],
   ['101011', '101'],
   ['101101', '011'],
   ['101110', '110'],
   ['110000', '000'],
   ['110011', '011'],
   ['110101', '110'],
   ['110110', '101']]),
 ('01',
  [['000001', '000'],
   ['000010', '000'],
   ['000100', '000'],
   ['000111', '000'],
   ['011001', '110'],
   ['011010', '000'],
   ['011100', '011'],
   ['011111', '101'],
   ['101001', '000'],
   ['101010', '101'],
   ['101100', '011'],
   ['101111', '110'],
   ['110001', '110'],
   ['110010', '101'],
   ['110100', '000'],
   ['110111', '011']]),
 ('10',
  [['001000', '000'],
   ['001011', '110'],
   ['001101', '000'],
   ['001110', '110'],
   ['010000', '000'],
   ['010011', '000'],
   ['010101', '101'],
   ['010110', '101'],
   ['100000', '000'],
   ['100011', '011'],
   ['100101', '011'],
   ['100110', '000'],
   [

In [24]:
list(ti.io_counts_dict.items())

[('00', [7, 0, 0, 3, 0, 3, 3, 0]),
 ('01', [7, 0, 0, 3, 0, 3, 3, 0]),
 ('10', [7, 0, 0, 3, 0, 3, 3, 0]),
 ('11', [0, 5, 5, 0, 5, 0, 0, 1])]

In [25]:
ti.i_share_vars

In [26]:
print(tabulate(list(ti.io_shares_dict['11']), headers=['a1a2a3b1b2b3', 'p1p2p3']))

  a1a2a3b1b2b3    p1p2p3
--------------  --------
        001001       010
        001010       100
        001100       010
        001111       100
        010001       100
        010010       100
        010100       001
        010111       001
        100001       010
        100010       001
        100100       001
        100111       010
        111001       100
        111010       001
        111100       010
        111111       111
